## Reduction operation: the sum of the numbers in the range [0, value)

In [1]:
import numpy as np

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Secuencial

value = 5*10**4

X = np.random.rand(value)

# Para imprimir los pimeros valores del array

# print(X[0:12])

# Utilizando las operaciones mágicas de ipython

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("Time taken by reduction operation using a function:", tiempo)


print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")


# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")


# Utilizando numpy.ndarray.sum()

tiempo= %timeit -r 2 -o -q X.sum()

print("Time taken by reduction operation using numpy.ndarray.sum():", tiempo)

print("Now, the result using numpy.ndarray.sum():", X.sum())

Time taken by reduction operation using a function: 5.41 ms ± 273 µs per loop (mean ± std. dev. of 2 runs, 100 loops each)
And the result of the sum of numbers in the range [0, value) is: 25038.001204452554

Time taken by reduction operation using numpy.sum(): 13.9 µs ± 24.3 ns per loop (mean ± std. dev. of 2 runs, 100,000 loops each)
Now, the result using numpy.sum(): 25038.001204452543 
 
Time taken by reduction operation using numpy.ndarray.sum(): 12 µs ± 26.8 ns per loop (mean ± std. dev. of 2 runs, 100,000 loops each)
Now, the result using numpy.ndarray.sum(): 25038.001204452543


In [ ]:
a) Librería cupy: En la siguiente celda de código del notebook9 vamos a utilizar el paquete cupy para
acelerar dicha operación de reducción. Como se ha explicado, la libreria cupy es una librería muy
similar a la librería numpy específicamente diseñada para GPUs. De hecho, la mayoría de funciones
que hay en numpy tienen el mismo nombre en cupy. Por tanto, de las 2 formas de hacer la suma de
los elementos del array, por medio de la función reduc_operation y por medio de la función sum
de la librería numpy, vamos a usar únicamente la función sum de la librería cupy.
Lo que tienes que hacer es modificar el notebook para crear el array en la GPU (usando las funciones
de la librería cupy análogas a las de la librería numpy) y utilizar la función sum para calcular la suma
de los elementos del array. Como la GPU ya es paralela, no tienes que paralelizar nada más.

In [2]:
import cupy as cp

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Secuencial

value = 5*10**4

# Crear el array en la GPU
X = cp.random.rand(value)

# Para imprimir los primeros valores del array
# print(X[0:12])

# Utilizando las operaciones mágicas de ipython
tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("Time taken by reduction operation using a function:", tiempo)
print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")

# Utilizando cupy.sum() para realizar la suma en la GPU
tiempo = %timeit -r 2 -o -q cp.sum(X)

print("Time taken by reduction operation using cupy.sum():", tiempo)
print("Now, the result using cupy.sum():", cp.sum(X),"\n ")

# Utilizando cupy.ndarray.sum()
tiempo = %timeit -r 2 -o -q X.sum()

print("Time taken by reduction operation using cupy.ndarray.sum():", tiempo)
print("Now, the result using cupy.ndarray.sum():", X.sum())

Time taken by reduction operation using a function: 502 ms ± 1.17 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 24946.204564209584

Time taken by reduction operation using cupy.sum(): 17 µs ± 60.2 ns per loop (mean ± std. dev. of 2 runs, 100,000 loops each)
Now, the result using cupy.sum(): 24946.2045642094 
 
Time taken by reduction operation using cupy.ndarray.sum(): 16.1 µs ± 6.68 ns per loop (mean ± std. dev. of 2 runs, 100,000 loops each)
Now, the result using cupy.ndarray.sum(): 24946.2045642094


In [ ]:
b) Librería Numba: En la siguiente celda de código del notebook10 vamos a utilizar el paquete Numba
para acelerar dicha operación de reducción. Como se ha explicado, la libreria Numba te permite crear
ufuncs muy similares a las de la librería numpy que se pueden ejecutar en la GPU.
Lo que tienes que hacer es crear una ufunc que te permita hacer la reducción del array de forma
análoga a las de la librería numpy, y utilizar la función sum para calcular la suma de los elementos
del array. Como la GPU ya es paralela, no tienes que paralelizar nada más.

In [3]:
import cupy as cp
from numba import cuda

# Creación de la ufunc de suma utilizando Numba
@cuda.jit
def sum_gpu(arr, out):
    start = cuda.grid(1)
    stride = cuda.gridsize(1)
    tmp = 0
    for i in range(start, arr.size, stride):
        tmp += arr[i]
    cuda.atomic.add(out, 0, tmp)

# Secuencial

value = 5*10**4

# Crear el array en la GPU
X = cp.random.rand(value)

# Crear un array de salida para la suma (lo inicializamos en 0)
out = cp.zeros(1, dtype=cp.float64)

# Ejecutar la ufunc sum_gpu en la GPU
threads_per_block = 128
blocks_per_grid = (X.size + (threads_per_block - 1)) // threads_per_block

# Llamamos a la ufunc con la cantidad de bloques y hilos adecuados
sum_gpu[blocks_per_grid, threads_per_block](X, out)

# Ahora out[0] contiene la suma
print(f"Sum of the array using Numba ufunc: {out[0]}")

Sum of the array using Numba ufunc: 25020.613475582693


In [6]:
import sys
import cupy as cp
from numba import cuda

# Comprobar si estamos en un entorno de ejecución interactivo (Jupyter) o ejecutando desde la línea de comandos
if '__file__' in globals():  # Estamos ejecutando el archivo como un script (por ejemplo, en SLURM)
    if len(sys.argv) > 1:
        value = int(sys.argv[1])
    else:
        value = 5 * 10**4  # Valor por defecto si no se pasa ningún argumento
else:  # Estamos en un entorno interactivo como Jupyter
    value = 5 * 10**4  # Valor por defecto o ajustado manualmente en Jupyter

print(f"Number of elements: {value}")

# Creación de la ufunc de suma utilizando Numba
@cuda.jit
def sum_gpu(arr, out):
    start = cuda.grid(1)
    stride = cuda.gridsize(1)
    tmp = 0
    for i in range(start, arr.size, stride):
        tmp += arr[i]
    cuda.atomic.add(out, 0, tmp)

# Crear el array en la GPU
X = cp.random.rand(value)

# Crear un array de salida para la suma (lo inicializamos en 0)
out = cp.zeros(1, dtype=cp.float64)

# Ejecutar la ufunc sum_gpu en la GPU
threads_per_block = 128
blocks_per_grid = (X.size + (threads_per_block - 1)) // threads_per_block

# Llamamos a la ufunc con la cantidad de bloques y hilos adecuados
sum_gpu[blocks_per_grid, threads_per_block](X, out)

# Ahora out[0] contiene la suma
print(f"Sum of the array using Numba ufunc: {out[0]}")

Number of elements: 50000
Sum of the array using Numba ufunc: 25016.60663032525


In [ ]:
d) Crea una nueva celda de texto debajo de la última celda de código para explicar los resultados
obtenidos por los paquetes cupy y Numba usando la GPU.

In [ ]:
1. Operación de Reducción en CPU (Usando numpy y una función personalizada)

In [ ]:
Tiempo de ejecución utilizando la función personalizada: 5.41 ms (± 273 µs)
Resultado de la suma: 25038.0012044525
Tiempo de ejecución utilizando numpy.sum(): 13.9 µs (± 24.3 ns)
Resultado usando numpy.sum(): 25038.001204452543
Tiempo de ejecución utilizando numpy.ndarray.sum(): 12 µs (± 26.8 ns)
Resultado usando numpy.ndarray.sum(): 25038.001204452543

In [ ]:
La función personalizada de suma (reduc_operation) es más lenta porque no aprovecha las optimizaciones de numpy para operaciones vectorizadas. 
Utiliza un bucle for en la CPU, lo cual es menos eficiente.
Al usar numpy.sum() y numpy.ndarray.sum(), los tiempos de ejecución son significativamente más rápidos, ya que estas funciones están optimizadas 
internamente para operaciones en masa (vectorizadas).

In [ ]:
2. Operación de Reducción en GPU (Usando cupy)

In [ ]:
Tiempo de ejecución utilizando la función personalizada de reducción: 502 ms (± 1.17 ms)
Resultado de la suma: 24946.2045642095
Tiempo de ejecución utilizando cupy.sum(): 17 µs (± 60.2 ns)
Resultado usando cupy.sum(): 24946.2045642094
Tiempo de ejecución utilizando cupy.ndarray.sum(): 16.1 µs (± 6.68 ns)
Resultado usando cupy.ndarray.sum(): 24946.2045642094

In [ ]:
La función personalizada (reduc_operation) en la GPU es mucho más lenta que las versiones optimizadas de cupy.sum() y cupy.ndarray.sum(). 
Esto se debe a que la función personalizada no está aprovechando completamente las capacidades paralelas de la GPU. Aunque se ejecuta en la GPU, 
el código secuencial dentro de un bucle no es eficiente.
Las funciones cupy.sum() y cupy.ndarray.sum() están altamente optimizadas para la GPU, lo que resulta en tiempos de ejecución mucho más rápidos, 
incluso a un tamaño de array de 50,000 elementos.

In [ ]:
3. Operación de Reducción en GPU (Usando numba)